In [1]:
import os
import warnings
from math import sqrt
warnings.filterwarnings("ignore")
os.chdir('/home/yash/Documents/Decision-Flip-Experiments')
from scipy.misc import imread
import matplotlib.patches as mpatches
from models import *
from plotter import *
from saveloader import *
from fgsm_cifar import fgsm
from fgsm_cifar_wrt_class_flipstop import fgsm_wrt_class
from helper import *
from helper import _to_categorical
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
sd = 'shape_dict'

X_train, y_train, X_test, y_test, X_valid, y_valid = load_data(os)

class Dummy:
    pass

env = Dummy()

Loading CIFAR10
Loading pre-Shuffled training data


In [2]:
# We need a scope since the inference graph will be reused later
with tf.variable_scope('model'):
    
    #target-model
    env.x = tf.placeholder(tf.float32, (None, img_rows, img_cols,  img_chas), name='x')
    env.y = tf.placeholder(tf.float32, (None, n_classes), name='y')
    env.training = tf.placeholder(bool, (), name='mode')
    conv0 = tf.layers.conv2d(env.x, filters=32, kernel_size=[3, 3],
                             padding='same', name='conv0',
                             activation=tf.nn.relu)
    pool0 = tf.layers.max_pooling2d(conv0, pool_size=[2, 2],
                                    strides=2, name='pool0')
    conv1 = tf.layers.conv2d(pool0, filters=64,
                             kernel_size=[3, 3], padding='same',
                             name='conv1', activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(conv1, pool_size=[2, 2],
                                    strides=2, name='pool1')
    conv2 = tf.layers.conv2d(pool1, filters=128,
                             kernel_size=[1, 1], padding='same',
                             name='conv2', activation=tf.nn.relu)
    flat = tf.reshape(conv2, [-1, 8 * 8 * 128], name='flatten')
    logits = tf.layers.dense(flat, units=10, name='logits')

    env.ybar = tf.nn.softmax(logits, name='ybar')
    
    #finding accuracy 
    z = tf.argmax(env.y, axis=1)
    zbar = tf.argmax(env.ybar, axis=1)
    env.count = tf.cast(tf.equal(z, zbar), tf.float32)
    env.acc = tf.reduce_mean(env.count, name='acc')
    
    #finding loss 
    xent = tf.nn.softmax_cross_entropy_with_logits_v2(labels=env.y,logits=logits)
    env.loss = tf.reduce_mean(xent, name='loss')
    
    #finding activations
    env.activations = [tf.reshape(env.x, [-1]), tf.reshape(pool0, [-1]), tf.reshape(pool1, [-1]), 
                      tf.reshape(flat, [-1]), tf.reshape(logits, [-1])]
    
    #finding activation gradients
    grads_x = tf.reshape(tf.gradients(env.loss, env.x), [32*32*3]) #32x32x3
    grads_xl1 = tf.reshape(tf.gradients(env.loss, pool0), [16*16*32]) #16x16x32
    grads_xl2 = tf.reshape(tf.gradients(env.loss, pool1), [ 8*8*64]) #8x8x64
    grads_xl3 = tf.reshape(tf.gradients(env.loss, flat), [ 8192]) #8192
    grads_y = tf.reshape(tf.gradients(env.loss, logits), [ 10]) #10
    
    # [[32,32,3], [16,16,32], [8,8,64], 8192, 10]
    env.grads_activations = [grads_x, grads_xl1, grads_xl2, grads_xl3, grads_y]
    
    #finding layer gradients    
    l1_vars = tf.get_default_graph().get_tensor_by_name(os.path.split(conv0.name)[0] + '/kernel:0')
    grads_l1 = tf.reshape(tf.gradients(env.loss, l1_vars), [ 27*32])
    l2_vars = tf.get_default_graph().get_tensor_by_name(os.path.split(conv1.name)[0] + '/kernel:0')
    grads_l2 = tf.reshape(tf.gradients(env.loss, l2_vars), [ 9*32*64])
    l3_vars = tf.get_default_graph().get_tensor_by_name(os.path.split(conv2.name)[0] + '/kernel:0')
    grads_l3 = tf.reshape(tf.gradients(env.loss, l3_vars), [ 64*128])
    l4_vars = tf.get_default_graph().get_tensor_by_name(os.path.split(logits.name)[0] + '/kernel:0')
    grads_l4 = tf.reshape(tf.gradients(env.loss, l4_vars), [ 8192*10])
    
    # [[3,3,3,32], [3,3,32,64], [1,1,64,128], [8192,10]]
    env.grads_model = [grads_l1, grads_l2, grads_l3, grads_l4]
    
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.control_dependencies(extra_update_ops):
    env.optim = tf.train.AdamOptimizer(beta1=0.9, beta2=0.999, epsilon=1e-08,).minimize(env.loss)

with tf.variable_scope('sub_model_1'):
    inp = 3072
    shape_m1 = [inp, int(sqrt(inp)), 2]
    env.x_m1, env.y_m1, env.optimizer_m1, env.preds_m1, env.acc_m1 = sub_attack_model(shape_m1)

with tf.variable_scope('sub_model_2'):
    inp = 8192
    shape_m2 = [inp, int(sqrt(inp)), 2]
    env.x_m2, env.y_m2, env.optimizer_m2, env.preds_m2, env.acc_m2 = sub_attack_model(shape_m2)

with tf.variable_scope('sub_model_3'):
    inp = 4096
    shape_m3 = [inp, int(sqrt(inp)), 2]
    env.x_m3, env.y_m3, env.optimizer_m3, env.preds_m3, env.acc_m3 = sub_attack_model(shape_m3)

with tf.variable_scope('sub_model_4'):
    inp = 8192
    shape_m4 = [inp, int(sqrt(inp)), 2]
    env.x_m4, env.y_m4, env.optimizer_m4, env.preds_m4, env.acc_m4 = sub_attack_model(shape_m4)

with tf.variable_scope('sub_model_5'):
    inp = 10
    shape_m5 = [inp, int(sqrt(inp)), 2]
    env.x_m5, env.y_m5, env.optimizer_m5, env.preds_m5, env.acc_m5 = sub_attack_model(shape_m5)

with tf.variable_scope('sub_model_6'):
    inp = 3072
    shape_m6 = [inp, int(sqrt(inp)), 2]
    env.x_m6, env.y_m6, env.optimizer_m6, env.preds_m6, env.acc_m6 = sub_attack_model(shape_m6)

with tf.variable_scope('sub_model_7'):
    inp = 8192
    shape_m7 = [inp, int(sqrt(inp)), 2]
    env.x_m7, env.y_m7, env.optimizer_m7, env.preds_m7, env.acc_m7 = sub_attack_model(shape_m7)

with tf.variable_scope('sub_model_8'):
    inp = 4096
    shape_m8 = [inp, int(sqrt(inp)), 2]
    env.x_m8, env.y_m8, env.optimizer_m8, env.preds_m8, env.acc_m8 = sub_attack_model(shape_m8)

with tf.variable_scope('sub_model_9'):
    inp = 8192
    shape_m9 = [inp, int(sqrt(inp)), 2]
    env.x_m9, env.y_m9, env.optimizer_m9, env.preds_m9, env.acc_m9 = sub_attack_model(shape_m9)

with tf.variable_scope('sub_model_10'):
    inp = 10
    shape_m10 = [inp, int(sqrt(inp)), 2]
    env.x_m10, env.y_m10, env.optimizer_m10, env.preds_m10, env.acc_m10 = sub_attack_model(shape_m10)

with tf.variable_scope('sub_model_11'):
    inp = 864
    shape_m11 = [inp, int(sqrt(inp)), 2]
    env.x_m11, env.y_m11, env.optimizer_m11, env.preds_m11, env.acc_m11 = sub_attack_model(shape_m11)

with tf.variable_scope('sub_model_12'):
    inp = 18432
    shape_m12 = [inp, int(sqrt(inp)), 2]
    env.x_m12, env.y_m12, env.optimizer_m12, env.preds_m12, env.acc_m12 = sub_attack_model(shape_m12)

with tf.variable_scope('sub_model_13'):
    inp = 8192
    shape_m13 = [inp, int(sqrt(inp)), 2]
    env.x_m13, env.y_m13, env.optimizer_m13, env.preds_m13, env.acc_m13 = sub_attack_model(shape_m13)

with tf.variable_scope('sub_model_14'):
    inp = 81920
    shape_m14 = [inp, int(sqrt(inp)), 2]
    env.x_m14, env.y_m14, env.optimizer_m14, env.preds_m14, env.acc_m14 = sub_attack_model(shape_m14)


In [3]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

In [4]:
def save_model(label):
    saver = tf.train.Saver()
    saver.save(sess,  './models/cifar/' + label)
    

def restore_model(label):
    saver = tf.train.Saver()
    saver.restore(sess, './models/cifar/' + label)

In [5]:
def _evaluate(X_data, y_data, env):
    print('\nEvaluating')
    n_sample = X_data.shape[0]
    batch_size = 128
    n_batch = int(np.ceil(n_sample/batch_size))
    loss, acc = 0, 0
    ns = 0
    for ind in range(n_batch):
        print(' batch {0}/{1}'.format(ind+1, n_batch), end='\r')
        start = ind*batch_size
        end = min(n_sample, start+batch_size)
        batch_loss, batch_count, batch_acc = sess.run(
            [env.loss, env.count, env.acc],
            feed_dict={env.x: X_data[start:end],
                       env.y: y_data[start:end],
                       env.training: False})
        loss += batch_loss*batch_size
        ns+=batch_size
        acc += batch_acc*batch_size
    loss /= ns
    acc /= ns
    print(' loss: {0:.4f} acc: {1:.4f}'.format(loss, acc))
    return loss, acc

def _evaluate_submodel(i, X_data, y_data):
    print('Evaluating')
    n_sample = X_data.shape[0]
    batch_size = 128
    n_batch = int(np.ceil(n_sample/batch_size))
    loss, acc = 0, 0
    ns = 0
    
    attr_x = "x_m" + str(i)
    attr_y = "y_m" + str(i)
    attr_optim = "optimizer_m" + str(i)
    attr_acc = "acc_m" + str(i)
    attr_preds = "preds_m" + str(i)
    
    for ind in range(n_batch):
#         print(' batch {0}/{1}'.format(ind+1, n_batch), end='\r')
        start = ind*batch_size
        end = min(n_sample, start+batch_size)
        batch_acc = sess.run(getattr(env, attr_acc),
            feed_dict={getattr(env, attr_x): X_data[start:end],
                       getattr(env, attr_y): y_data[start:end],
                       env.training: False})
        ns+=batch_size
        acc += batch_acc*batch_size
    acc /= ns
    print('acc: {0:.4f}'.format(acc))
    return acc

In [18]:
def _predict(X_data, env):
    print('\nPredicting')
    n_sample = X_data.shape[0]
    batch_size = 128
    n_batch = int(np.ceil(n_sample/batch_size))
    yval = np.empty((X_data.shape[0], n_classes))
    for ind in range(n_batch):
        print(' batch {0}/{1}'.format(ind+1, n_batch), end='\r')
        start = ind*batch_size
        end = min(n_sample, start+batch_size)
        batch_y = sess.run(env.ybar, feed_dict={
            env.x: X_data[start:end], env.training: False})
        yval[start:end] = batch_y
    return yval

def train(label = 'generalized_model', n_epoch = 20, batch_size = 128):
    print('\nTraining')
    n_sample = X_train.shape[0]
    n_batch = int(np.ceil(n_sample/batch_size))
    for epoch in range(n_epoch):
        print('Epoch {0}/{1}'.format(epoch+1, n_epoch))
        for ind in range(n_batch):
            print(' batch {0}/{1}'.format(ind+1, n_batch), end='\r')
            start = ind*batch_size
            end = min(n_sample, start+batch_size)
            sess.run(env.optim, feed_dict={env.x: X_train[start:end],
                                           env.y: y_train[start:end],
                                           env.training: True})
        if((epoch+1)%10 == 0):
            model_label = label+ '{0}'.format(epoch)
            _, train_acc = _evaluate(X_train, y_train, env)
            print("Train Acc. :" + str(train_acc))
            save_model(model_label)
            
    save_model(label)

def train_again(X, y, epochs):
    #Not making batches, do that if size > 128
    for i in range(X.shape[0]):
        for e in range(epochs):
            sess.run(env.optim, feed_dict={env.x: [X[i]],
                                           env.y: [y[i]],
                                           env.training: True})
            
    
    

#send i+1
def train_submodel(i, X, y, n_epoch = 100, batch_size = 128, disp_freq=10):
    print('\n Training ' + str(i) + 'th model.')
    n_samples = X.shape[0]
    n_batch = int(np.ceil(n_samples/batch_size))
    
    attr_x = "x_m" + str(i)
    attr_y = "y_m" + str(i)
    attr_optim = "optimizer_m" + str(i)
    attr_acc = "acc_m" + str(i)
    attr_preds = "preds_m" + str(i)
    
    for epoch in range(n_epoch):
        print('Epoch {0}/{1}'.format(epoch+1, n_epoch), end = '\r')
        
        for ind in range(n_batch):
#             print(' batch {0}/{1}'.format(ind+1, n_batch), end='\r')
            start = ind*batch_size
            end = min(n_samples, start+batch_size)
            feed_dict = {getattr(env, attr_x): X[start:end], getattr(env, attr_y): y[start:end], env.training: True}
            sess.run(getattr(env, attr_optim), feed_dict)
            
        if((epoch+1)%disp_freq == 0):
            train_acc = _evaluate_submodel(i, X, y)
    


In [7]:
def create_flip_wrt_class(X, Y, label = None):
    print('\nCrafting adversarial')
    n_sample = X.shape[0]
    pred = np.argmax(Y,axis=1)
    batch_size = 1
    n_batch = int(np.ceil(n_sample/batch_size))
    n_epoch = 20
    x_adv_shape = list(X.shape)[1:]
    x_adv_shape = np.append(np.append(n_sample,n_classes),x_adv_shape)
    X_adv = np.empty(x_adv_shape)
    for ind in range(n_batch):
        print(' batch {0}/{1}'.format(ind+1, n_batch), end='\r')
        start = ind*batch_size
        end = min(n_sample, start+batch_size)
        tmp = sess.run(env.x_adv_wrt_class, feed_dict={env.x: X[start:end],
                                             env.y: Y[start:end],
                                             env.training: False})
        tmp[pred[start]] = X[start]
        X_adv[start:end] = tmp
    if(label != None):
        print('\nSaving adversarial')
        os.makedirs('data', exist_ok=True)
        save_as_txt(get_flip_path(label), X_adv)
    return X_adv    

def get_signals(X,y):
    activation_grads = []
    model_grads = []
    activations = []
    for i in range(len(X)):
        acts, ag, mg = sess.run([env. activations, env.grads_activations, env.grads_model], 
                          feed_dict={env.x: [X[i]], env.y: [y[i]]})
        activations.append(acts)
        activation_grads.append(ag)
        model_grads.append(mg)
        
    return np.array(activations), np.array(activation_grads), np.array(model_grads)

# create a batchwise call in build_submodelD if required! 
def build_submodelX(X,y):
    activations, activation_grads, model_grads = get_signals(X, y)
    aX, agX, mgX = [],[],[]
    
    for i in range (activations.shape[1]):
        aX.append([item[i] for item in activations])
    
    for i in range (activation_grads.shape[1]):
        agX.append([item[i] for item in activation_grads])
    
    for i in range (model_grads.shape[1]):
        mgX.append([item[i] for item in model_grads])
    
    return aX+ agX+ mgX

def merge(a,b):
    if(len(a) == 0):
        return b
    for i in range(len(a)):
        a[i] = a[i] + b[i]
    return a
    

def build_submodelD():
    #taking 5000 test and 5000 train images
    n = 1000
    sy = np.array([[0,1]]*n + [[1,0]]*n, np.float32)
    X = np.array(list(X_test[:n]) + list(X_train[:n]))
    y = np.array(list(y_test[:n]) + list(y_test[:n]))
    batch_size = 64
    n_samples = X.shape[0]
    n_batch = int(np.ceil(n_samples/batch_size))
    
    sX=[]
    for ind in range(n_batch):
        print(' batch {0}/{1}'.format(ind+1, n_batch), end='\r')
        start = ind*batch_size
        end = min(n_samples, start+batch_size)
        tsX = build_submodelX(X[start:end], y[start:end])
        sX = merge(sX, tsX)
    return sX, sy
    
    

    

In [12]:
# train(n_epoch=1)
label = "my-model"
# save_model(label)
restore_model(label)
sX, sy = build_submodelD()

for i in range(len(sX)):
    train_submodel(i+1, np.array(sX[i]), sy)



INFO:tensorflow:Restoring parameters from ./models/cifar/my-model
 batch 32/32
 Training 1th model.
Evaluating0
acc: 0.51176
Evaluating0
acc: 0.51176

 Training 2th model.
Evaluating0
acc: 0.51176
Evaluating0
acc: 0.51176

 Training 3th model.
Evaluating0
acc: 0.51176


KeyboardInterrupt: 

In [ ]:
for i in range(len(sX)):
    print(len(sX[i][0]))

In [ ]:
a

In [ ]:
# #training m1

# feed_dict={env.x_m1: imgs, env.y_m1: imgs_y, env.training: True}
# sess.run(getattr(env, 'optimizer_m1'), feed_dict)
# print(sess.run(getattr(env, 'acc_m1'), feed_dict))


In [21]:
imgs = X_train[:4000]
imgs = np.reshape(imgs, [-1, 3072])
imgs_y = np.array([[0,1]]*2000 + [[1,0]]*2000, np.float32)

np.random.seed(42)
ind = np.arange(0, imgs.shape[0])
ind = np.random.shuffle(ind)
imgs, imgs_y = imgs[ind][0], imgs_y[ind][0]

train_submodel(1, imgs, imgs_y, n_epoch= 100)


 Training 1th model.
Evaluating00
acc: 0.51172
Evaluating00
acc: 0.51172
Evaluating00
acc: 0.51172
Evaluating00
acc: 0.51172
Evaluating00
acc: 0.51172
Evaluating00
acc: 0.51172
Evaluating00
acc: 0.51172
Evaluating00
acc: 0.51172
Evaluating00
acc: 0.51172
Evaluating100
acc: 0.51172


In [ ]:
for i in range(1, 15):
    print("with tf.variable_scope('sub_model_{0}'):\n    inp =\n    shape_m{0} = [inp, int(sqrt(inp)), 2]\n    env.x_m{0}, env.y_m{0}, env.optimizer_m{0}, env.preds_m{0}, env.acc_m{0} = sub_attack_model(shape_m{0})\n".format(i))

In [ ]:
en = [activations[0], activation_grads[0], model_grads[0]]
print (len(en))
print (len(en[0]))


In [ ]:
activation_grads = [item[0] for item in ag] # gets the first item in each sublist.

print(len(activation_grads))
print(len(activation_grads[0]))
print(len(activation_grads[0][0]))
print(len(activation_grads[0][0][0]))
# print(len(activation_grads[4]))

In [ ]:
# def main_func(X,y,model_label):
#     restore_model(model_label)
#     Xflip = create_flip_wrt_class(X,y)
#     return Xflip


# n = 10
# model_label="cifar_with_cnn" + str(200)
# X = X_train[:n]
# y = y_train[:n]
# X_flip = main_func(X,y,model_label)

# flip_label = "flipped_examples"
# save_as_txt(get_flip_path(flip_label), X_flip)
# X_flip_load = load_from_txt(get_flip_path(flip_label))

# # print(X_flip == X_flip_load)

In [ ]:
obj = Dummy()

obj.hi = "yash"
obj.bye = "yash"

a = "bye"
print(getattr(obj,a))